<a href="https://colab.research.google.com/github/quasar-in-the-mist/Plate-Tectonics/blob/main/Game_theory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Game Rules
# Objective: Reduce the opponent's Health Points (HP) from 20 to 0. If a player tries to draw from an empty deck, they lose.
# Card Types:
# Monsters: Placed on the field. They have Attack (ATK) and Defense (DEF). During the combat phase, they attack. If the opponent has monsters, they attack a random monster. If the attacker's ATK is strictly greater than the defender's DEF, the defender is destroyed. If the opponent has no monsters, they attack the opponent's HP directly.
# Spells: Played directly from the hand, trigger an automated effect (healing, direct damage, destroying cards), and are then discarded.
# Turn Flow: Draw a card ➡️ Play a random card ➡️ Resolve Effects ➡️ Combat Phase.


# The Library: create_card_library() instantiates 10 unique cards. Every time a player builds a deck, they randomly grab 15 cards from this library.
# The Hand & Board: The Player class tracks a deck array, hand array, and field array.
# Automated Decisions: When play_turn is called, the script forces the player to select a random card from their hand list.
# If it's a Monster, it goes to the field list.
# If it's a Spell, it hits the resolve_spell logic block, mutates the state (changing HP, deleting items from the opponent's field array, etc.), and disappears.
# Combat: The combat_phase loops through every monster on the active player's field and simulates attacks automatically based on the presence of enemy monsters

In [ ]:
import random
import time

# --- CARD AND LIBRARY SETUP ---
class Card:
    def __init__(self, name, card_type, atk=0, dfn=0, effect=None):
        self.name = name
        self.card_type = card_type  # 'Monster' or 'Spell'
        self.atk = atk
        self.dfn = dfn
        self.effect = effect  # String identifying the spell/special effect

    def __str__(self):
        if self.card_type == "Monster":
            return f"[{self.name} (ATK: {self.atk} | DEF: {self.dfn})]"
        return f"[{self.name} (Spell)]"

def create_card_library():
    """Returns 10 unique cards to build decks from."""
    return [
        Card("Goblin Bruiser", "Monster", atk=3, dfn=2),
        Card("Shield Knight", "Monster", atk=1, dfn=5),
        Card("Berserker", "Monster", atk=5, dfn=1),
        Card("Wall of Ice", "Monster", atk=0, dfn=6),
        Card("The Reaper", "Monster", atk=4, dfn=4),
        Card("Vampire Bat", "Monster", atk=2, dfn=1, effect="lifesteal"),
        Card("Fireball", "Spell", effect="fireball"),       # 3 damage to opponent
        Card("Healing Potion", "Spell", effect="heal"),     # Heals 5 HP
        Card("Lightning Strike", "Spell", effect="smite"),  # Destroys random enemy monster
        Card("Sword of Power", "Spell", effect="buff")      # +2 ATK to a random friendly monster
    ]

# --- PLAYER SETUP ---
class Player:
    def __init__(self, name):
        self.name = name
        self.hp = 20
        self.deck = []
        self.hand = []
        self.field = []

    def build_deck(self, library):
        # Create a 15-card deck by randomly sampling from the library
        self.deck = [random.choice(library) for _ in range(15)]
        random.shuffle(self.deck)

    def draw(self):
        if not self.deck:
            return False # Deck out
        card = self.deck.pop()
        self.hand.append(card)
        print(f"🎴 {self.name} draws {card.name}.")
        return True

# --- GAME ENGINE ---
class Game:
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
        self.turn_count = 1

    def resolve_spell(self, card, caster, opponent):
        """Automated state changes based on Spell effects."""
        if card.effect == "fireball":
            print(f"🔥 {caster.name} casts {card.name}! Deals 3 damage to {opponent.name}.")
            opponent.hp -= 3
        elif card.effect == "heal":
            print(f"💖 {caster.name} casts {card.name}! Heals for 5 HP.")
            caster.hp += 5
        elif card.effect == "smite":
            if opponent.field:
                target = random.choice(opponent.field)
                opponent.field.remove(target)
                print(f"⚡ {caster.name} casts {card.name}! It destroys {target.name}.")
            else:
                print(f"⚡ {caster.name} casts {card.name}, but {opponent.name} has no monsters. It fizzles!")
        elif card.effect == "buff":
            if caster.field:
                target = random.choice(caster.field)
                target.atk += 2
                print(f"⚔️ {caster.name} casts {card.name}! {target.name} gains +2 ATK (Now {target.atk}).")
            else:
                print(f"⚔️ {caster.name} casts {card.name}, but has no monsters to buff. It fizzles!")

    def combat_phase(self, attacker, defender):
        """Automated combat for all monsters on the attacker's field."""
        for monster in attacker.field:
            # Special Monster Effect: Lifesteal
            if monster.effect == "lifesteal":
                attacker.hp += monster.atk
                print(f"🦇 {monster.name} uses lifesteal! {attacker.name} heals {monster.atk} HP.")

            if defender.field:
                # Attack a random monster
                target = random.choice(defender.field)
                print(f"⚔️ {monster.name} (ATK: {monster.atk}) attacks {target.name} (DEF: {target.dfn})!")

                if monster.atk > target.dfn:
                    print(f"   💥 {target.name} is destroyed!")
                    defender.field.remove(target)
                else:
                    print(f"   🛡️ {target.name} defends successfully.")
            else:
                # Direct Attack
                print(f"🗡️ {monster.name} attacks {defender.name} directly for {monster.atk} damage!")
                defender.hp -= monster.atk

    def play_turn(self, active_player, opponent):
        print(f"\n--- Turn {self.turn_count}: {active_player.name}'s Turn ---")
        print(f"HP -> {active_player.name}: {active_player.hp} | {opponent.name}: {opponent.hp}")

        # 1. Draw Phase
        if not active_player.draw():
            print(f"💀 {active_player.name} tried to draw from an empty deck and loses!")
            active_player.hp = 0
            return

        # 2. Main Phase (Play a random card)
        if active_player.hand:
            card_to_play = random.choice(active_player.hand)
            active_player.hand.remove(card_to_play)

            if card_to_play.card_type == "Monster":
                active_player.field.append(card_to_play)
                print(f"summoned 👾 {card_to_play} to the field.")
            elif card_to_play.card_type == "Spell":
                self.resolve_spell(card_to_play, active_player, opponent)

        # 3. Combat Phase
        if active_player.field:
            self.combat_phase(active_player, opponent)

    def start_game(self):
        print("🚀 Starting Automated Card Game Simulation!\n")

        # Initial Draw (3 cards each)
        for _ in range(3):
            self.p1.draw()
            self.p2.draw()

        # Game Loop
        while self.p1.hp > 0 and self.p2.hp > 0:
            # Player 1 turn
            self.play_turn(self.p1, self.p2)
            if self.p2.hp <= 0 or self.p1.hp <= 0:
                break

            time.sleep(1) # Slight pause for readability in terminal

            # Player 2 turn
            self.play_turn(self.p2, self.p1)
            self.turn_count += 1

            time.sleep(1)

        # Game Over Resolution
        print("\n" + "="*30)
        print("🏆 GAME OVER 🏆")
        if self.p1.hp > 0:
            print(f"{self.p1.name} wins with {self.p1.hp} HP remaining!")
        elif self.p2.hp > 0:
            print(f"{self.p2.name} wins with {self.p2.hp} HP remaining!")
        else:
            print("It's a draw!")
        print("="*30)

# --- EXECUTION ---
if __name__ == "__main__":
    library = create_card_library()

    player1 = Player("Yugi")
    player2 = Player("Kaiba")

    player1.build_deck(library)
    player2.build_deck(library)

    game = Game(player1, player2)
    game.start_game()